In [1]:
import numpy as np
import torch as th


from glob import glob
from utils import *
import ray
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
ray.init(num_cpus=160, ignore_reinit_error=True)

2023-08-30 10:08:16,850	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.11.3
Ray version:,2.6.2
Dashboard:,http://127.0.0.1:8265


In [3]:
insmp = Backtester(20190103, 20210101, pattern = "./checkpoints/*sat*/",)
otsmp = Backtester(20210101, 20211231, pattern = "./checkpoints/*sat*/",)

num of factors:180
num of factors:180


In [4]:
factors = insmp.calc_factor()
factors2 = otsmp.calc_factor()

calc_factor took 347 seconds
calc_factor took 185 seconds


In [5]:
df_ic = insmp.calc_ic()
df_ic2 = otsmp.calc_ic()

calc_ic took 576 seconds
calc_ic took 325 seconds


In [6]:
otsmp.df_ic.query("y1>0.01 and y2>0.01 and y5>0.01 and @insmp.df_ic.y1>.01 and @insmp.df_ic.y2>.015 and @insmp.df_ic.y5>.02")

,y1,y2,y5
20230811171800_satd_lexpr08_lopt34_10_0042,0.029813,0.032791,0.039772
20230813143935_satd_lexpr08_lopt34_10_0322,0.011760,0.013922,0.012883
20230813145451_satd_lexpr08_lopt34_10_0856,0.026628,0.030035,0.037093
20230813145458_satd_lexpr08_lopt34_10_0262,0.023027,0.024977,0.034143
20230813145501_satd_lexpr08_lopt34_10_0726,0.029509,0.029523,0.031023
...,...,...,...
20230829165951_satd_lexpr20_lopt50_10_2307,0.017191,0.018013,0.019074
20230829165954_satd_lexpr20_lopt50_10_0028,0.029554,0.031833,0.035371
20230829165955_satd_lexpr20_lopt50_10_0561,0.015328,0.017678,0.019975
20230829171842_satd_lexpr20_lopt50_10_3566,0.010570,0.012524,0.012202


In [7]:
selected_idx = otsmp.df_ic.query("y1>0.01 and y2>0.01 and y5>0.01 and @insmp.df_ic.y1>.01 and @insmp.df_ic.y2>.015 and @insmp.df_ic.y5>.02").index

In [8]:
feature_dict = dict(zip(otsmp.factor_names,otsmp.factors))

In [9]:
selected_sigs = [feature_dict[i] for i in selected_idx]

In [10]:
def calc_sig_corr(sigs):
    n = len(sigs)
    df_corr = np.ones((n,n))
    corrs = []
    for i in range(n):
        for j in range(i+1,n):
            ic = torch.nanmean(batch_pearsonr(sigs[i],sigs[j],))
            df_corr[i,j] = ic
            df_corr[j,i] = ic
    return df_corr
            

In [11]:
df_corr = calc_sig_corr(selected_sigs)

In [12]:
df_corr.shape

(128, 128)

In [13]:
df_corr = pd.DataFrame(df_corr)

In [14]:
lowcorr_idx = remove_high_corr_factors(df_corr.values)
print(len(lowcorr_idx))

95


In [15]:
final = [selected_idx[idx] for idx in lowcorr_idx]

In [16]:

# 设置显示最大列数
pd.set_option('display.max_columns', None)

# 设置显示最大行数
pd.set_option('display.max_rows', None)
finaldf =  otsmp.df_ic.loc[final]

In [17]:
print(finaldf)

                                                  y1        y2        y5
20230811171800_satd_lexpr08_lopt34_10_0042  0.029813  0.032791  0.039772
20230813143935_satd_lexpr08_lopt34_10_0322  0.011760  0.013922  0.012883
20230813145451_satd_lexpr08_lopt34_10_0856  0.026628  0.030035  0.037093
20230813145501_satd_lexpr08_lopt34_10_0726  0.029509  0.029523  0.031023
20230814092613_satd_lexpr08_lopt34_10_0788  0.030111  0.031117  0.033647
20230814092614_satd_lexpr08_lopt34_10_0177  0.010298  0.011240  0.015453
20230814092615_satd_lexpr08_lopt34_10_0018  0.021988  0.023256  0.023869
20230814092615_satd_lexpr08_lopt34_10_0143  0.022127  0.025572  0.027143
20230814102408_satd_lexpr15_lopt34_10_0319  0.026133  0.027363  0.035816
20230814175110_satd_lexpr15_lopt34_10_0269  0.025749  0.027213  0.033657
20230814175115_satd_lexpr15_lopt34_10_0852  0.035036  0.036162  0.041299
20230815095228_satd_lexpr10_lopt34_10_0762  0.021464  0.022482  0.027379
20230815095232_satd_lexpr10_lopt34_10_0094  0.03347

In [18]:
finaldf.describe()

,y1,y2,y5
count,95.000000,95.000000,95.000000
mean,0.022757,0.024558,0.027700
std,0.006209,0.006460,0.008193
min,0.010131,0.011148,0.011492
25%,0.017829,0.019307,0.020884
50%,0.022777,0.024574,0.028378
75%,0.026722,0.029100,0.034043
max,0.036356,0.038088,0.041299


In [19]:
fs  = selected_sigs
y1d = otsmp.calter.ret1d
y2d = otsmp.calter.ret2d
y5d = otsmp.calter.ret5d

In [20]:
q1d = [batch_topk(fs[i],y1d) for i in range(len(fs))]
q2d = [batch_topk(fs[i],y2d) for i in range(len(fs))]
q5d = [batch_topk(fs[i],y5d) for i in range(len(fs))]

In [ ]:
q1d = [q1d[idx] for idx in lowcorr_idx]
q2d = [q2d[idx] for idx in lowcorr_idx]
q5d = [q5d[idx] for idx  in lowcorr_idx]

In [ ]:
finaldf['q1d'] = q1d
finaldf['q2d'] = q2d
finaldf['q5d'] = q5d

In [ ]:
finaldf

,y1,y2,y5,q1d,q2d,q5d
20230828110829_satd_lexpr20_lopt50_10_3402,0.021288,0.022038,0.022402,"(0.00042, 6e-05)","(0.00072, 0.00024)","(0.00141, 0.00072)"
20230828110840_satd_lexpr20_lopt50_10_1995,0.022279,0.023746,0.032066,"(9e-05, -0.0009)","(7e-05, -0.00129)","(0.00111, -0.00069)"
20230828110849_satd_lexpr20_lopt50_10_2167,0.022718,0.023017,0.024803,"(7e-05, -0.00028)","(0.00012, -0.00047)","(0.00011, -0.00075)"
20230828110853_satd_lexpr20_lopt50_10_8290,0.027974,0.029046,0.034231,"(0.00062, 0.00085)","(0.00107, 0.00118)","(0.00243, 0.00305)"


In [ ]:
# finaldf.to_csv('result.csv')

In [ ]:
# df_corr.iloc[lowcorr_idx,lowcorr_idx].to_csv('df_corr.csv')

In [26]:
df_corr.replace(1,0).describe()

,0,1,2,3
count,4.000000,4.000000,4.000000,4.000000
mean,0.263824,0.275655,0.281087,0.287631
std,0.178281,0.187343,0.188382,0.191851
min,0.000000,0.000000,0.000000,0.000000
25%,0.237301,0.237301,0.263332,0.281160
50%,0.333755,0.352131,0.362994,0.381333
75%,0.360278,0.390484,0.380750,0.387804
max,0.387786,0.398359,0.398359,0.387859
